<a href="https://colab.research.google.com/github/ching-wong/my_notebooks/blob/main/Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Models, Layers, and Loss Functions with TensorFlow - Week 2

How to define custom loss functions.

>[Custom Models, Layers, and Loss Functions with TensorFlow - Week 2](#scrollTo=VqW8A7V0q51z)

>>[Using loss functions](#scrollTo=GpV1fDf9gXlP)

>>[Define and use custom loss function](#scrollTo=kwgv-zCaQatF)

>>[Loss function as a class](#scrollTo=UTyJ0YJnUMfM)

>>[Example: Contrastive loss](#scrollTo=E8ntZsuUZnT3)



## Using loss functions

We usually specify which loss function we want to use in the compile function.

For example, we have a simple model with only one dense layer.

In [1]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

input = Input(shape=[1])
predictions = Dense(1, activation="softmax", name="dense")(input)

simple_model = Model(inputs = input, outputs = predictions)

In [2]:
# option 1: use a string
simple_model.compile(loss='mse', optimizer='sgd')

# option 2: import the class, which allows us to change the hyperparameters
from tensorflow.keras.losses import MeanSquaredError
simple_model.compile(loss=MeanSquaredError(reduction = "sum"), optimizer='sgd')

## Define and use custom loss function

A loss function should take two inputs: y_true and y_pred.

In [3]:
def custom_loss_function(y_true, y_pred):

  return abs(y_true-y_pred)

To compile, simply use the function we just defined.

In [4]:
simple_model.compile(loss=custom_loss_function, optimizer='sgd')

One can also set other parameters for the custom loss function by using a wrapper. In this example, we have para as a hyperparameter for the custom loss function.

In [5]:
def custom_loss_function_with_parameters(para = 3):

  def loss_function(y_true, y_pred):

    return abs(y_true-y_pred)*para

  return loss_function

To compile, we use the outer function with the parameters we want.

In [6]:
simple_model.compile(loss=custom_loss_function_with_parameters(para = 5), optimizer='sgd')

## Loss function as a class

Loss functions can be implemented as classes. It should be inherited from the Loss class.

In [7]:
from tensorflow.keras.losses import Loss

class MyLossClass(Loss):

  def __init__(self, para):
    super().__init__()
    self.para = para

  def call(self, y_true, y_pred):
    return abs(y_true-y_pred)*self.para

We can then compile the model as usual.

In [8]:
simple_model.compile(loss=MyLossClass(para = 5), optimizer='sgd')

## Example: Contrastive loss

In the Siamese network, we use the contrastive loss.

The essence is: Two images are similar, if and only if they produce similar feature vector.

The formula is:
$$\text{loss} = Y * D^2 + (1-Y) * \max(\text{margin} - D, 0)^2,$$

where

* $Y$ is the tensor, 1 if the images are similar, 0 if not,
* $D$ is the tensor, Euclidean distances between pair of images,
* margin is a constant, distance threshold to determine similar or not.

When $Y = 1$, $\text{loss} = D^2$.

When $Y = 0$, $\text{loss} = \max(\text{margin} - D, 0)^2$.

$Y$ should be the y_true and $D$ should be y_pred.

Note: K.square(x) is element-wise squaring in TensorFlow tensors.


In [9]:
from tensorflow.keras import backend as K

def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss